<a href="https://colab.research.google.com/github/fsilvino/INE5644-data-mining-projeto-final/blob/master/eda/eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd

df = pd.read_csv('../datasets/openDataSUS/INFLUD-21-09-2020.csv', delimiter=';')
df.head()

d:\github\ine5644-data-mining-projeto-final\venv\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (15,60,62,63,64,92,94,108,117,121,123) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,ID_UNIDADE,...,DS_AN_OUT,TP_AM_SOR,SOR_OUT,DT_CO_SOR,TP_SOR,OUT_SOR,DT_RES,RES_IGG,RES_IGM,RES_IGA
0,08/01/2020,2,05/01/2020,2,SP,GVE XVII CAMPINAS,1342.0,CAMPINAS,350950,HOSPITAL E MATERNIDADE CELSO PIERRO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28/01/2020,5,22/01/2020,4,PR,02RS METROPOLITANA,1356.0,CURITIBA,410690,COMPLEXO HOSPITAL DE CLINICAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13/01/2020,3,11/01/2020,2,PR,16RS APUCARANA,1370.0,JANDAIA DO SUL,411210,CASA DE SAUDE E MAT N SENHORA FATIMA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18/02/2020,8,12/02/2020,7,MA,REGI0NAL DE SAUDE METROPOLITANA,1430.0,SAO LUIS,211130,SOCORRAO II,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19/02/2020,8,14/02/2020,7,PR,15RS MARINGA,1369.0,MARINGA,411520,HOSPITAL MUNICIPAL DE MARINGA THELMA VILLANOVA...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df.describe()

,SEM_NOT,SEM_PRI,CO_REGIONA,CO_MUN_NOT,CO_UNI_NOT,NU_IDADE_N,TP_IDADE,CS_GESTANT,CS_RACA,CS_ESCOL_N,...,AN_PARA1,AN_PARA2,AN_PARA3,AN_ADENO,AN_OUTRO,TP_AM_SOR,TP_SOR,RES_IGG,RES_IGM,RES_IGA
count,754901.000000,754901.000000,659567.000000,754901.000000,7.549010e+05,754901.000000,754901.000000,754901.000000,690162.000000,530223.000000,...,15.0,1.0,4.0,28.0,21321.0,47167.000000,43307.000000,50044.000000,51742.000000,32246.000000
mean,25.933997,24.670170,1572.204149,333623.722345,3.341047e+06,56.162698,2.975862,5.806508,3.628593,5.481228,...,1.0,1.0,1.0,1.0,1.0,1.906545,1.282425,2.255096,2.090487,4.174192
std,6.942949,6.963992,702.059583,91586.426839,2.439110e+06,22.622275,0.175956,0.927888,2.832437,3.482329,...,0.0,NaN,0.0,0.0,0.0,2.383618,0.819010,1.556260,1.478412,1.475100
min,1.000000,1.000000,1331.000000,110001.000000,1.900000e+01,-1.000000,1.000000,0.000000,1.000000,0.000000,...,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000
25%,21.000000,19.000000,1342.000000,280290.000000,2.081164e+06,42.000000,3.000000,5.000000,1.000000,2.000000,...,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,1.000000,1.000000,4.000000
50%,26.000000,25.000000,1430.000000,350950.000000,2.481286e+06,59.000000,3.000000,6.000000,4.000000,5.000000,...,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,2.000000,1.000000,4.000000
75%,31.000000,30.000000,1519.000000,355030.000000,4.028155e+06,73.000000,3.000000,6.000000,4.000000,9.000000,...,1.0,1.0,1.0,1.0,1.0,1.000000,1.000000,4.000000,4.000000,4.000000
max,39.000000,39.000000,6256.000000,530010.000000,9.997091e+06,148.000000,3.000000,9.000000,9.000000,9.000000,...,1.0,1.0,1.0,1.0,1.0,9.000000,4.000000,9.000000,9.000000,9.000000
